In [16]:
import os

import pandas as pd
import numpy as np
from icecream import ic
from tqdm import tqdm

from event_triggered_response import *

In [17]:
multiscope_experiment_table = pd.read_csv('multiscope_experiment_table_filtered.csv', index_col='ophys_experiment_id').reset_index().drop(columns = ['Unnamed: 0'])
multiscope_experiment_table.head()

,ophys_experiment_id,ophys_session_id,ophys_container_id,mouse_id,cre_line,session_type,imaging_depth,experience_level
0,951980473,951410079,1018028345,457841,Sst-IRES-Cre,OPHYS_1_images_A,225,Familiar
1,953659743,952430817,1018028345,457841,Sst-IRES-Cre,OPHYS_2_images_A_passive,225,Familiar
2,958527471,954954402,1018028345,457841,Sst-IRES-Cre,OPHYS_3_images_A,225,Familiar
3,956941844,955775716,1018028345,457841,Sst-IRES-Cre,OPHYS_3_images_A,225,Familiar
4,957759564,957020350,1018028345,457841,Sst-IRES-Cre,OPHYS_4_images_B,225,Novel 1


In [18]:
if not os.path.exists('mean_cell_etr_data'):
    os.mkdir('mean_cell_etr_data')

all_cell_etr_files = os.listdir('cell_etr_data')

all_experiments = []
all_cells = []

for filename in all_cell_etr_files:
    split = filename.replace('.csv', '').split('_')
    all_experiments.append(int(split[0]))
    all_cells.append(int(split[1]))

all_experiments = list(set(all_experiments))

for id in tqdm(all_experiments):
    one_experiment = [x for x in all_cell_etr_files if str(x).find(str(id)) == 0]
    one_experiment_etr = []
    for x in one_experiment:
        one_etr = pd.read_csv(f'cell_etr_data/{x}').drop(columns = ['Unnamed: 0'])
        one_experiment_etr.append(one_etr)
    one_experiment_etr = pd.concat(one_experiment_etr)
    one_experiment_etr.groupby(['ophys_experiment_id', 'cell_specimen_id', 'time'])['dff'].mean()\
        .reset_index()\
        .merge(multiscope_experiment_table, on = 'ophys_experiment_id', how = 'left')\
        .to_csv(f'mean_cell_etr_data/{id}.csv')

100%|██████████| 25/25 [02:47<00:00,  6.69s/it]
